In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('1 дз taxi_dataset.csv')
df2 = pd.read_csv('2 ДЗ taxi_dataset_with_predictions.csv')[['id', 'trip_duration']]
df = df.merge(df2, on='id').set_index('id')
df.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
id,,,,,,,,,,
id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455.0
id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663.0
id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124.0
id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429.0
id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435.0


Описание данных:
- id - ID поездки
- vendor_id - ID компании, осуществляющей перевозку
- pickup_datetime - Таймкод начала поездки
- dropoff_datetime - Таймкод конца поездки
- passenger_count - Количество пассажиров
- pickup_longitude - Долгота точки, в которой началась поездка
- pickup_latitude - Широта точки, в которой началась поездка
- dropoff_longitude - Долгота точки, в которой закончилась поездка
- dropoff_latitude - Широта точки, в которой закончилась поездка
- store_and_fwd_flag- Yes/No: Была ли информация сохранена в памяти транспортного средства из-за потери соединения с сервером


Попробуем построить модель Линейной Регрессии!

В качестве Выборки используйте таргетную колонку (*trip_duration*) и все признаки, кроме времени начала заказа (*pikcup_datetime*)


In [3]:
model = LinearRegression()
model.fit(df.drop(['pickup_datetime', 
                   'dropoff_datetime', 
                   'trip_duration', 
                   'store_and_fwd_flag'], axis=1), 
          df['trip_duration'])

LinearRegression()

In [4]:
# коэффициенты beta
model.coef_

array([  203.40290932,    10.65643273,  2557.33580766, -3707.27380547,
        -618.70312991, -1705.04256923])

In [5]:
# b0
model.intercept_

364595.3769216642

Теперь реализуйте функцию LinearRegressionByMatrix, которая будет принимать на вход 3 параметра:

Матрицу объект-признак **(X)**, вектор с ответами **(Y)**, булевый параметр **fit_intercept**, цель которого - добавить константный признак (состоящий из единичек), если True, и ничего не делать, если False.

Функция должна вернуть одномерный np.array объект с оцененными **$\beta_1, ..., \beta_n, \beta_0$**

Реализуйте решение через аналитическое решение в матрицах из лекции и практики:

$$
\beta^* = (X^T \cdot X)^{-1} \cdot X^T \cdot Y
$$

In [29]:
def LinearRegressionByMatrix(X, Y, fit_intercept=True):
    def round_3(x):
        return round(x, 3)
    if fit_intercept == True:
        X['const'] = 1
    X, Y = X.values, Y.values
    x_t_x = np.dot(X.T, X)
    betas = np.dot(np.dot(np.linalg.inv(x_t_x), X.T), Y)
    return list(map(round_3, betas.tolist()))

In [30]:
LinearRegressionByMatrix(df.drop(['pickup_datetime', 
                                   'dropoff_datetime', 
                                   'trip_duration', 
                                   'store_and_fwd_flag'], axis=1), 
                         df['trip_duration']
                        )

[203.403, 10.656, 2557.336, -3707.274, -618.703, -1705.043, 364595.376]

In [31]:
betas = LinearRegressionByMatrix(df.drop(['pickup_datetime', 
                                   'dropoff_datetime', 
                                   'trip_duration', 
                                   'store_and_fwd_flag'], axis=1), 
                         df['trip_duration']
                        )

In [38]:
df['conts'] = 1
X = df.drop(['pickup_datetime', 
            'dropoff_datetime', 
            'trip_duration', 
            'store_and_fwd_flag'], axis=1).values

Имея коэффициенты модели, можно восстановить предсказания для каждого объекта!

Сделайте это через операцию матричного произведения матрицы **X** и полученных коэффициентов (как результат работы *LinearRegressionByMatrix*)

In [39]:
np.dot(X, np.array(betas))

array([ 932.73230035,  922.97427775, 1075.39164201, ..., 1112.33307194,
        820.17291492,  647.27693969])